In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

driver_path = 'C:/Users/Landon Hawley/Downloads/chromedriver-win64/chromedriver.exe'
service = Service(driver_path)

driver = webdriver.Chrome(service=service)

# Open the URL
url = "https://www.espn.com/nfl/stats/player/_/season/2024/seasontype/2"
driver.get(url)

time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

button = driver.find_element(By.LINK_TEXT, "Show More")
button.click()

# Allow time for the page to load
time.sleep(5)

# Get the page source after JavaScript has loaded
html_content = driver.page_source

driver.quit()

In [2]:
soup = BeautifulSoup(html_content, "lxml")

namerank_columns = ['Rank', 'Name/Team']
namerank_df = pd.DataFrame(columns = namerank_columns)

namerank_div = soup.find(name = 'div', attrs = {'class' : 'ResponsiveTable ResponsiveTable--fixed-left mt4 Table2__title--remove-capitalization'})
table = namerank_div.find("table") 
tbody = table.find("tbody")  # Get the tbody element
rows = tbody.find_all("tr")  # Find all table rows

for row in rows:
    cols = row.find_all("td")  # Get all table data cells
    data = [col.text.strip() for col in cols]  # Extract text from each column
    namerank_series = pd.Series(data, index = namerank_columns)
    namerank_df = pd.concat([namerank_df, namerank_series.to_frame().T], ignore_index=True)

In [3]:
import re


team_abbrs = [
    "ARI", "ATL", "BAL", "BUF", "CAR", "CHI", "CIN", "CLE", "DAL", "DEN",
    "DET", "GB", "HOU", "IND", "JAX", "KC", "LAC", "LAR", "LV", "MIA",
    "MIN", "NE", "NO", "NYG", "NYJ", "PHI", "PIT", "SEA", "SF", "TB",
    "TEN", "WSH", "OAK"
]


team_pattern = r'(' + '|'.join(team_abbrs) + r')$'


def split_string_first(s):
    match = re.search(team_pattern, s)
    if match:
        return s[:match.start()]
    return s


def split_string_second(s):
    match = re.search(team_pattern, s)
    if match:
        return match.group()
    return ''


namerank_df['Name'] = namerank_df['Name/Team'].apply(split_string_first)
namerank_df['Team'] = namerank_df['Name/Team'].apply(split_string_second)
namerank_df.drop('Name/Team', inplace=True, axis=1)

In [4]:
stats_columns = ['Position', 'Games Played', 'Completions', 'Passing Attempts', 'Completion Percentage', 'Passing Yards',
                 'Avg Yards per Pass Attempt', 'Passing Yards per Game', 'Longest Pass', 'Passing Touchdowns', 'Interceptions',
                 'Total Sacks', 'Sack Yards Lost', 'Adjusted QBR', 'Passer Rating']
stats_df = pd.DataFrame(columns = stats_columns)

stats_div = soup.find(name = 'div', attrs = {'class' : 'Table__Scroller'})
table = stats_div.find("table") 
tbody = table.find("tbody")  # Get the tbody element
rows = tbody.find_all("tr")  # Find all table rows

for row in rows:
    cols = row.find_all("td")  # Get all table data cells
    data = [col.text.strip() for col in cols]  # Extract text from each column
    stats_series = pd.Series(data, index = stats_columns)
    stats_df = pd.concat([stats_df, stats_series.to_frame().T], ignore_index=True)

stats_df

,Position,Games Played,Completions,Passing Attempts,Completion Percentage,Passing Yards,Avg Yards per Pass Attempt,Passing Yards per Game,Longest Pass,Passing Touchdowns,Interceptions,Total Sacks,Sack Yards Lost,Adjusted QBR,Passer Rating
0,QB,17,460,652,70.6,"4,918",7.5,289.3,70,43,9,48,278,74.7,108.5
1,QB,17,390,539,72.4,"4,629",8.6,272.3,82,37,12,31,234,68.4,111.8
2,QB,17,407,570,71.4,"4,500",7.9,264.7,57,41,16,40,248,61.6,106.8
3,QB,17,407,578,70.4,"4,320",7.5,254.1,71,21,15,50,338,53.8,93.2
4,QB,17,361,545,66.2,"4,319",7.9,254.1,97,35,12,48,335,60.4,102.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,WR,17,0,1,0.0,0,0.0,0.0,0,0,0,0,0,1.7,39.6
105,WR,16,0,1,0.0,0,0.0,0.0,0,0,0,0,0,5.4,39.6
106,OT,17,0,0,0.0,0,0.0,0.0,0,0,0,1,1,1.9,39.6
107,WR,11,0,1,0.0,0,0.0,0.0,0,0,0,0,0,2.1,39.6


In [5]:
total_stats = pd.concat([namerank_df, stats_df], axis=1)
total_stats

,Rank,Name,Team,Position,Games Played,Completions,Passing Attempts,Completion Percentage,Passing Yards,Avg Yards per Pass Attempt,Passing Yards per Game,Longest Pass,Passing Touchdowns,Interceptions,Total Sacks,Sack Yards Lost,Adjusted QBR,Passer Rating
0,1,Joe Burrow,CIN,QB,17,460,652,70.6,"4,918",7.5,289.3,70,43,9,48,278,74.7,108.5
1,2,Jared Goff,DET,QB,17,390,539,72.4,"4,629",8.6,272.3,82,37,12,31,234,68.4,111.8
2,3,Baker Mayfield,TB,QB,17,407,570,71.4,"4,500",7.9,264.7,57,41,16,40,248,61.6,106.8
3,4,Geno Smith,SEA,QB,17,407,578,70.4,"4,320",7.5,254.1,71,21,15,50,338,53.8,93.2
4,5,Sam Darnold,MIN,QB,17,361,545,66.2,"4,319",7.9,254.1,97,35,12,48,335,60.4,102.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,91,Tutu Atwell,LAR,WR,17,0,1,0.0,0,0.0,0.0,0,0,0,0,0,1.7,39.6
105,91,Dyami Brown,WSH,WR,16,0,1,0.0,0,0.0,0.0,0,0,0,0,0,5.4,39.6
106,91,Penei Sewell,DET,OT,17,0,0,0.0,0,0.0,0.0,0,0,0,1,1,1.9,39.6
107,91,Ricky Pearsall,SF,WR,11,0,1,0.0,0,0.0,0.0,0,0,0,0,0,2.1,39.6


In [7]:
total_stats.drop(total_stats[total_stats['Name'].str.contains('/')].index, inplace = True)
total_stats

,Rank,Name,Team,Position,Games Played,Completions,Passing Attempts,Completion Percentage,Passing Yards,Avg Yards per Pass Attempt,Passing Yards per Game,Longest Pass,Passing Touchdowns,Interceptions,Total Sacks,Sack Yards Lost,Adjusted QBR,Passer Rating
0,1,Joe Burrow,CIN,QB,17,460,652,70.6,"4,918",7.5,289.3,70,43,9,48,278,74.7,108.5
1,2,Jared Goff,DET,QB,17,390,539,72.4,"4,629",8.6,272.3,82,37,12,31,234,68.4,111.8
2,3,Baker Mayfield,TB,QB,17,407,570,71.4,"4,500",7.9,264.7,57,41,16,40,248,61.6,106.8
3,4,Geno Smith,SEA,QB,17,407,578,70.4,"4,320",7.5,254.1,71,21,15,50,338,53.8,93.2
4,5,Sam Darnold,MIN,QB,17,361,545,66.2,"4,319",7.9,254.1,97,35,12,48,335,60.4,102.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,91,Tutu Atwell,LAR,WR,17,0,1,0.0,0,0.0,0.0,0,0,0,0,0,1.7,39.6
105,91,Dyami Brown,WSH,WR,16,0,1,0.0,0,0.0,0.0,0,0,0,0,0,5.4,39.6
106,91,Penei Sewell,DET,OT,17,0,0,0.0,0,0.0,0.0,0,0,0,1,1,1.9,39.6
107,91,Ricky Pearsall,SF,WR,11,0,1,0.0,0,0.0,0.0,0,0,0,0,0,2.1,39.6


In [9]:
total_stats.to_csv("Passing_Stats_2024.csv", index = False)
print("Data saved to 2024_Passing_Stats.csv")

Data saved to 2024_Passing_Stats.csv
